In [ ]:
# 한글 불러옴
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

!apt-get update -qq
!apt-get install fonts-nanum* -qq

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 1s (18.9 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 121925 files and direc

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import geopandas as gpd
from shapely.geometry import Point

# 나눔고딕 폰트 설정
font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
font_prop = fm.FontProperties(fname=font_path, size=12)
plt.rc('font', family='NanumGothic')

# 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 버스 정류소 데이터

## 위치 데이터 전처리

In [ ]:
df_busstop = pd.read_csv('/content/drive/MyDrive/[통계청 공모전] 음주운전/1 데이터/소현_유동인구 상권 및 시설 위치/대중교통(지하철역, 버스정류장) 위치/서울시 버스정류소 위치정보.csv', encoding = 'euc-kr')
df_busstop = df_busstop[['정류소명', 'X좌표', 'Y좌표']]
df_busstop.head()

,정류소명,X좌표,Y좌표
0,종로2가사거리,126.987752,37.569808
1,창경궁.서울대학교병원,126.996522,37.579433
2,명륜3가.성대입구,126.998251,37.582581
3,종로2가.삼일교,126.987613,37.568579
4,혜화동로터리.여운형활동터,127.001744,37.586243


## 행정동 데이터

In [ ]:
gdf_dong = gpd.read_file('/content/drive/MyDrive/[통계청 공모전] 음주운전/1 데이터/소현_유동인구 상권 및 시설 위치/행정동경계.geojson')

In [ ]:
df_busstop['geometry'] = df_busstop.apply(lambda row: Point([row['X좌표'], row['Y좌표']]), axis=1)
gdf_points = gpd.GeoDataFrame(df_busstop, geometry='geometry')

In [ ]:
gdf_points.crs = {'init': 'epsg:4326'}
gdf_dong = gdf_dong.to_crs(epsg=4326)

/usr/local/lib/python3.10/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [ ]:
gdf_result = gpd.sjoin(gdf_points, gdf_dong, how='left', op = 'within')

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3473: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):
<ipython-input-7-ff38215f2427>:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:4326 +type=crs
Right CRS: EPSG:4326

  gdf_result = gpd.sjoin(gdf_points, gdf_dong, how='left', op = 'within')


In [ ]:
print(gdf_result[['X좌표', 'Y좌표', 'temp', 'adm_cd2']])

              X좌표        Y좌표             temp     adm_cd2
0      126.987752  37.569808  종로구 종로1·2·3·4가동  1111061500
1      126.996522  37.579433  종로구 종로1·2·3·4가동  1111061500
2      126.998251  37.582581          종로구 혜화동  1111065000
3      126.987613  37.568579  종로구 종로1·2·3·4가동  1111061500
4      127.001744  37.586243          종로구 혜화동  1111065000
...           ...        ...              ...         ...
11285  127.139339  37.550386         강동구 천호1동  1174060000
11286  127.140046  37.550643         강동구 천호1동  1174060000
11287  127.123596  37.533630         강동구 성내2동  1174065000
11288  127.125497  37.536155         강동구 성내2동  1174065000
11289  127.127337  37.540343         강동구 천호2동  1174061000

[11290 rows x 4 columns]


In [ ]:
df_busstop['행정동명'] = gdf_result['temp']
df_busstop['행정동 코드'] = gdf_result['adm_cd2']
df_busstop.head()

,정류소명,X좌표,Y좌표,geometry,행정동명,행정동 코드
0,종로2가사거리,126.987752,37.569808,POINT (126.987752 37.569808),종로구 종로1·2·3·4가동,1111061500
1,창경궁.서울대학교병원,126.996522,37.579433,POINT (126.996522 37.579433),종로구 종로1·2·3·4가동,1111061500
2,명륜3가.성대입구,126.998251,37.582581,POINT (126.998251 37.582581),종로구 혜화동,1111065000
3,종로2가.삼일교,126.987613,37.568579,POINT (126.9876130976 37.5685792736),종로구 종로1·2·3·4가동,1111061500
4,혜화동로터리.여운형활동터,127.001744,37.586243,POINT (127.001744 37.586243),종로구 혜화동,1111065000


## 행정동별 집계

In [ ]:
df_busstop_count = df_busstop.groupby('행정동명').count()
df_busstop_count.reset_index(inplace=True)
df_busstop_count = df_busstop_count[['행정동명', '정류소명']]
df_busstop_count.rename(columns={'정류소명': '버스 정류소 개수'}, inplace=True)
df_busstop_count.head()

,행정동명,버스 정류소 개수
0,강남구 개포1동,15
1,강남구 개포2동,34
2,강남구 개포3동,11
3,강남구 개포4동,28
4,강남구 논현1동,17


In [ ]:
def find_code(row, df_busstop):
  match = df_busstop[df_busstop['행정동명'] == row['행정동명']]
  if not match.empty:
    return match['행정동 코드'].values[0]
  else:
    return None

df_busstop_count['행정동 코드'] = df_busstop_count.apply(lambda row: find_code(row, df_busstop), axis=1)
df_busstop_count.head()

,행정동명,버스 정류소 개수,행정동 코드
0,강남구 개포1동,15,1168066000
1,강남구 개포2동,34,1168067000
2,강남구 개포3동,11,1168067500
3,강남구 개포4동,28,1168069000
4,강남구 논현1동,17,1168052100


In [ ]:
df_busstop_count = df_busstop_count[df_busstop_count['행정동 코드'].str.startswith('11')]
df_busstop_count.head()

,행정동명,버스 정류소 개수,행정동 코드
0,강남구 개포1동,15,1168066000
1,강남구 개포2동,34,1168067000
2,강남구 개포3동,11,1168067500
3,강남구 개포4동,28,1168069000
4,강남구 논현1동,17,1168052100


In [ ]:
df_busstop_count.to_csv('/content/drive/MyDrive/[통계청 공모전] 음주운전/1 데이터/소현_유동인구 상권 및 시설 위치/대중교통(지하철역, 버스정류장) 위치/행정동별 버스 정류장 개수.csv')

# 지하철역 데이터

## 위치 데이터 전처리

In [ ]:
df_subway = pd.read_csv('/content/drive/MyDrive/[통계청 공모전] 음주운전/1 데이터/소현_유동인구 상권 및 시설 위치/대중교통(지하철역, 버스정류장) 위치/서울시 지하철역 엘리베이터 위치정보.csv', encoding = 'euc-kr')
df_subway = df_subway[['노드 WKT', '시군구명', '읍면동명', '지하철역명']]
df_subway.dropna(subset= ['지하철역명'], inplace= True)
df_subway.reset_index(drop=True, inplace=True)
df_subway.head()

,노드 WKT,시군구명,읍면동명,지하철역명
0,POINT(127.00225226116126 37.58169863717817),종로구,동숭동,혜화
1,POINT(127.00904036229613 37.57079391329676),종로구,종로6가,동대문
2,POINT(127.00175612966481 37.58144803112957),종로구,명륜4가,혜화
3,POINT(127.01744971746365 37.57329704981851),종로구,숭인동,동묘앞
4,POINT(127.01551499790813 37.57951303710329),종로구,숭인동,창신


In [ ]:
df_subway = df_subway.drop_duplicates(subset='지하철역명')
df_subway.reset_index(drop=True, inplace=True)
df_subway.head()

,노드 WKT,시군구명,읍면동명,지하철역명
0,POINT(127.00225226116126 37.58169863717817),종로구,동숭동,혜화
1,POINT(127.00904036229613 37.57079391329676),종로구,종로6가,동대문
2,POINT(127.01744971746365 37.57329704981851),종로구,숭인동,동묘앞
3,POINT(127.01551499790813 37.57951303710329),종로구,숭인동,창신
4,POINT(127.02297735704515 37.57595176742774),종로구,숭인동,신설동


In [ ]:
def extract_coordinates(row):
  coordinates = row.replace('POINT(', '').replace(')', '')
  x, y = coordinates.split(' ')
  return (float(x), float(y))

df_subway['경도'], df_subway['위도'] = zip(*df_subway['노드 WKT'].apply(extract_coordinates))
df_subway.head()

,노드 WKT,시군구명,읍면동명,지하철역명,경도,위도
0,POINT(127.00225226116126 37.58169863717817),종로구,동숭동,혜화,127.002252,37.581699
1,POINT(127.00904036229613 37.57079391329676),종로구,종로6가,동대문,127.009040,37.570794
2,POINT(127.01744971746365 37.57329704981851),종로구,숭인동,동묘앞,127.017450,37.573297
3,POINT(127.01551499790813 37.57951303710329),종로구,숭인동,창신,127.015515,37.579513
4,POINT(127.02297735704515 37.57595176742774),종로구,숭인동,신설동,127.022977,37.575952


## 행정동 데이터

In [ ]:
gdf_dong = gpd.read_file('/content/drive/MyDrive/[통계청 공모전] 음주운전/1 데이터/소현_유동인구 상권 및 시설 위치/행정동경계.geojson')

In [ ]:
df_subway['geometry'] = df_subway.apply(lambda row: Point([row['경도'], row['위도']]), axis=1)
gdf_points = gpd.GeoDataFrame(df_subway, geometry='geometry')

In [ ]:
gdf_points.crs = {'init': 'epsg:4326'}
gdf_dong = gdf_dong.to_crs(epsg=4326)

/usr/local/lib/python3.10/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [ ]:
gdf_result = gpd.sjoin(gdf_points, gdf_dong, how='left', op = 'within')

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3473: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):
<ipython-input-20-ff38215f2427>:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:4326 +type=crs
Right CRS: EPSG:4326

  gdf_result = gpd.sjoin(gdf_points, gdf_dong, how='left', op = 'within')


In [ ]:
print(gdf_result[['위도', '경도', 'temp', 'adm_cd2']])

            위도          경도         temp     adm_cd2
0    37.581699  127.002252      종로구 이화동  1111064000
1    37.570794  127.009040  종로구 종로5·6가동  1111063000
2    37.573297  127.017450     종로구 숭인2동  1111071000
3    37.579513  127.015515     종로구 숭인1동  1111070000
4    37.575952  127.022977     종로구 숭인2동  1111071000
..         ...         ...          ...         ...
266  37.535912  127.132907     강동구 성내2동  1174065000
267  37.554881  127.153833     강동구 명일2동  1174054000
268  37.537821  127.139783       강동구 길동  1174068500
269  37.538115  127.124487     강동구 성내2동  1174065000
270  37.527920  127.135963     강동구 성내3동  1174066000

[271 rows x 4 columns]


In [ ]:
df_subway['행정동명'] = gdf_result['temp']
df_subway['행정동 코드'] = gdf_result['adm_cd2']
df_subway.head()

,노드 WKT,시군구명,읍면동명,지하철역명,경도,위도,geometry,행정동명,행정동 코드
0,POINT(127.00225226116126 37.58169863717817),종로구,동숭동,혜화,127.002252,37.581699,POINT (127.00225226116126 37.58169863717817),종로구 이화동,1111064000
1,POINT(127.00904036229613 37.57079391329676),종로구,종로6가,동대문,127.009040,37.570794,POINT (127.00904036229613 37.57079391329676),종로구 종로5·6가동,1111063000
2,POINT(127.01744971746365 37.57329704981851),종로구,숭인동,동묘앞,127.017450,37.573297,POINT (127.01744971746365 37.57329704981851),종로구 숭인2동,1111071000
3,POINT(127.01551499790813 37.57951303710329),종로구,숭인동,창신,127.015515,37.579513,POINT (127.01551499790813 37.57951303710329),종로구 숭인1동,1111070000
4,POINT(127.02297735704515 37.57595176742774),종로구,숭인동,신설동,127.022977,37.575952,POINT (127.02297735704515 37.57595176742774),종로구 숭인2동,1111071000


## 행정동별 집계

In [ ]:
df_subway_count = df_subway.groupby('행정동명').count()
df_subway_count.reset_index(inplace=True)
df_subway_count = df_subway_count[['행정동명', '지하철역명']]
df_subway_count.rename(columns={'지하철역명': '지하철역 개수'}, inplace=True)
df_subway_count.head()

,행정동명,지하철역 개수
0,강남구 개포2동,3
1,강남구 개포3동,1
2,강남구 논현1동,2
3,강남구 논현2동,2
4,강남구 대치1동,3


In [ ]:
def find_code(row, df_subway):
  match = df_subway[df_subway['행정동명'] == row['행정동명']]
  if not match.empty:
    return match['행정동 코드'].values[0]
  else:
    return None

df_subway_count['행정동 코드'] = df_subway_count.apply(lambda row: find_code(row, df_subway), axis=1)
df_subway_count.head()

,행정동명,지하철역 개수,행정동 코드
0,강남구 개포2동,3,1168067000
1,강남구 개포3동,1,1168067500
2,강남구 논현1동,2,1168052100
3,강남구 논현2동,2,1168053100
4,강남구 대치1동,3,1168060000


In [ ]:
df_subway_count.to_csv('/content/drive/MyDrive/[통계청 공모전] 음주운전/1 데이터/소현_유동인구 상권 및 시설 위치/대중교통(지하철역, 버스정류장) 위치/행정동별 지하철역 개수.csv')